In [1]:
%matplotlib inline
%config InlineBackend.figure_format = 'svg'

import pandas as pd
import warnings
import sys
import numpy as np
import scipy as sc
import random
from scipy import stats
from numpy import log10, sqrt

mydir = '/Users/kenlocey/GitHub/HACRP-HAIs/'
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

def def_display_df_misses(df):
    df = df[df['Payment Reduction Reproduced?'] == 0]
    items = ['file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
             'CAUTI Footnote', 'CAUTI W Z Score', 'CAUTI SIR W Z Score',
             'CDI Footnote', 'CDI W Z Score', 'CDI SIR W Z Score',
             'CLABSI Footnote', 'CLABSI W Z Score', 'CLABSI SIR W Z Score',
             'MRSA Footnote', 'MRSA W Z Score', 'MRSA SIR W Z Score',
             'SSI Footnote', 'SSI W Z Score', 'SSI SIR W Z Score',
             'PSI-90 Footnote', 'PSI-90 W Z Score', 'PSI-90 SIR W Z Score',
             'Total HAC Footnote', 'Total HAC Score', 'Total HAC Score (derived)',
             'Payment Reduction Footnote', 'Payment Reduction', 
             'Payment Reduction (derived)', 'Payment Reduction Reproduced?',
             ]
    return df.filter(items=items)


def Winsorize_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    p5 = np.nanpercentile(x2, 5)
    p95 = np.nanpercentile(x2, 95)
    WinScores = []
    
    for i, val in enumerate(x2):
        wz = float(WinZs[i])
        val = float(val)
        
        if np.isnan(wz) == True:
            WinScores.append(np.nan)
            
        elif np.isnan(wz) == False:
            if val >= p5 and val <= p95:
                WinScores.append(val)
            elif val < p5:
                WinScores.append(p5)
            elif val > p95:
                WinScores.append(p95)
            elif np.isnan(val) == True:
                #print('val:', val, '|', WinZs[i])
                WinScores.append(np.nan)
        
    
    return WinScores
        

def ZScore_it(x, WinZs):
    
    x2 = []
    for i, val in enumerate(x):
        wz = float(WinZs[i])
        val = float(val)
        if np.isnan(wz) == True:
            x2.append(np.nan)
        else:
            x2.append(val)
    
    x2 = np.array(x2)
    avg = np.nanmean(x2)
    std = np.nanstd(x2)
    zscores = (x2 - avg) / std
    return zscores


## Load HAC file

In [2]:
hac_df = pd.read_pickle(mydir + "data/CareCompare_data/CombinedFiles_HACRP/Facility.pkl")
hac_df = hac_df[hac_df['file_year'] == '2021']
hac_df = hac_df[hac_df['file_month'] == '10']

print('HACRP years:', sorted(hac_df['Fiscal Year'].unique()))

features = ['CAUTI Score', 'CLABSI Score', 'CDI Score', 'MRSA Score', 'SSI Score',
            'Total HAC Score', 'Domain 1 Score', 'AHRQ PSI-90 Score', 'Domain 2 Score',
            'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'CDI W Z Score', 
            'SSI W Z Score', 'PSI-90 W Z Score']
for f in features:
    hac_df[f] = hac_df[f].astype(str)
    hac_df[f] = hac_df[f].str.replace('*', '')
    hac_df[f] = pd.to_numeric(hac_df[f], errors='coerce')

hac_df.dropna(how='all', axis=1, inplace=True)
hac_df.head()

HACRP years: [2021]


,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Facility ID,Facility Name,Fiscal Year,HAI Measures End Date,HAI Measures Start Date,MRSA Footnote,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year
0,NaN,-0.4612,NaN,-0.3173,NaN,0.6029,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2021,12/31/2019,01/01/2018,NaN,-0.7886,06/30/2019,NaN,07/01/2017,-0.3375,No,NaN,NaN,-1.0731,AL,NaN,-0.3958,10,2021
1,NaN,0.5331,NaN,-0.6850,NaN,2.2542,010005,MARSHALL MEDICAL CENTERS,2021,12/31/2019,01/01/2018,NaN,2.2000,06/30/2019,NaN,07/01/2017,0.0881,Yes,NaN,NaN,-1.0620,AL,NaN,0.5547,10,2021
2,NaN,-0.1304,NaN,-0.5237,NaN,-0.3226,010006,NORTH ALABAMA MEDICAL CENTER,2021,12/31/2019,01/01/2018,NaN,0.2020,06/30/2019,NaN,07/01/2017,0.6333,No,NaN,NaN,-0.3705,AL,NaN,-0.0853,10,2021
3,NaN,-1.5517,NaN,-0.3689,5.0,NaN,010007,MIZELL MEMORIAL HOSPITAL,2021,12/31/2019,01/01/2018,5.0,NaN,06/30/2019,NaN,07/01/2017,-0.4690,No,NaN,5.0,NaN,AL,NaN,-0.7965,10,2021
4,5.0,NaN,5.0,NaN,5.0,NaN,010008,CRENSHAW COMMUNITY HOSPITAL,2021,12/31/2019,01/01/2018,5.0,NaN,06/30/2019,NaN,07/01/2017,0.0247,No,NaN,5.0,NaN,AL,NaN,0.0247,10,2021


## Format dates within the HAC file 

In [3]:

## Convert all other date data to default datetime

hac_df['HAI Measures Start Date'] = pd.to_datetime(hac_df['HAI Measures Start Date'])
hac_df.style.format({'HAI Measures Start Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df['HAI Measures End Date'] = pd.to_datetime(hac_df['HAI Measures End Date'])
hac_df.style.format({'HAI Measures End Date': lambda t: t.strftime("%Y-%m-%d")})

hac_df = pd.concat([hac_df], axis=0)
hac_df.sort_values(by='HAI Measures End Date', inplace=True)

hac_df.rename(columns={'HAI Measures Start Date': 'Start Date', 'HAI Measures End Date': 'End Date'}, inplace=True)


## Load and merge HAI files

In [4]:
##############################   CAUTI   ################################################

cauti_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CAUTI_Data.pkl")
cauti_df = cauti_df.filter(items=['Facility ID', 'CAUTI Urinary Catheter Days (ICUs + select wards)', 
                                  'CAUTI Observed Cases (ICUs + select wards)', 
                                  'CAUTI Predicted Cases (ICUs + select wards)', 
                                  'Start Date', 'End Date'], axis=1)

cauti_df.rename(columns={'CAUTI Urinary Catheter Days (ICUs + select wards)': 'CAUTI Urinary Catheter Days', 
                         'CAUTI Observed Cases (ICUs + select wards)': 'CAUTI Observed Cases', 
                         'CAUTI Predicted Cases (ICUs + select wards)': 'CAUTI Predicted Cases', 
                        }, inplace=True)

features = ['CAUTI Urinary Catheter Days', 'CAUTI Observed Cases', 'CAUTI Predicted Cases']
for f in features:
    cauti_df[f] = cauti_df[f].astype(str)
    cauti_df[f] = pd.to_numeric(cauti_df[f], errors='coerce')

cauti_df['Start Date'] = pd.to_datetime(cauti_df['Start Date'])
cauti_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df['End Date'] = pd.to_datetime(cauti_df['End Date'])
cauti_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
cauti_df = cauti_df[(cauti_df['Start Date'].isin(hac_df['Start Date'].unique())) | (cauti_df['End Date'].isin(hac_df['End Date'].unique()))]

cauti_df.head()


,Facility ID,CAUTI Urinary Catheter Days,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date
4,030117,1658.0,0.0,0.862,2019-01-01,2019-12-31
11,361311,31.0,0.0,0.021,2019-01-01,2019-12-31
29,520100,4335.0,1.0,3.230,2019-01-01,2019-12-31
31,520045,4616.0,2.0,4.877,2019-01-01,2019-12-31
39,15004F,NaN,NaN,NaN,2019-01-01,2019-12-31


In [5]:
##############################   CLABSI   ###############################################

clabsi_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CLABSI_Data.pkl")
clabsi_df = clabsi_df.filter(items=['Facility ID', 'CLABSI Device Days (ICUs + select wards)', 
                                    'CLABSI Observed Cases (ICUs + select wards)', 
                                    'CLABSI Predicted Cases (ICUs + select wards)', 
                                    'Start Date', 'End Date'], axis=1)

clabsi_df.rename(columns={'CLABSI Device Days (ICUs + select wards)': 'CLABSI Device Days', 
                          'CLABSI Observed Cases (ICUs + select wards)': 'CLABSI Observed Cases', 
                          'CLABSI Predicted Cases (ICUs + select wards)': 'CLABSI Predicted Cases', 
                        }, inplace=True)


features = ['CLABSI Device Days', 'CLABSI Observed Cases', 'CLABSI Predicted Cases']
for f in features:
    clabsi_df[f] = clabsi_df[f].astype(str)
    clabsi_df[f] = pd.to_numeric(clabsi_df[f], errors='coerce')

clabsi_df['Start Date'] = pd.to_datetime(clabsi_df['Start Date'])
clabsi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df['End Date'] = pd.to_datetime(clabsi_df['End Date'])
clabsi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})
clabsi_df = clabsi_df[(clabsi_df['Start Date'].isin(hac_df['Start Date'].unique())) | (clabsi_df['End Date'].isin(hac_df['End Date'].unique()))]

clabsi_df.head()

,Facility ID,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,Start Date,End Date
2,36007F,NaN,NaN,NaN,2019-01-01,2019-12-31
21,140127,1883.0,0.0,1.692,2019-01-01,2019-12-31
37,171339,234.0,0.0,0.064,2019-01-01,2019-12-31
50,020018,24.0,0.0,0.013,2019-01-01,2019-12-31
66,051323,15.0,0.0,0.005,2019-01-01,2019-12-31


In [6]:

##############################   MRSA   ###############################################

mrsa_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/MRSA_Data.pkl")
mrsa_df = mrsa_df.filter(items=['Facility ID', 'MRSA patient days', 
                                'MRSA Observed Cases', 'MRSA Predicted Cases', 
                                'Start Date', 'End Date'], axis=1)
mrsa_df['Start Date'] = pd.to_datetime(mrsa_df['Start Date'])
mrsa_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
mrsa_df['End Date'] = pd.to_datetime(mrsa_df['End Date'])
mrsa_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})

mrsa_df = mrsa_df[(mrsa_df['Start Date'].isin(hac_df['Start Date'].unique())) | (mrsa_df['End Date'].isin(hac_df['End Date'].unique()))]

##############################   CDI   ###############################################

cdi_df = pd.read_pickle(mydir + "1_preprocess_CareCompare_data/preprocessed_HAI_data/CDI_Data.pkl")
cdi_df = cdi_df.filter(items=['Facility ID', 'CDIFF patient days', 
                              'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
                              'Start Date', 'End Date'], axis=1)
cdi_df['Start Date'] = pd.to_datetime(cdi_df['Start Date'])
cdi_df.style.format({'Start Date': lambda t: t.strftime("%Y-%m-%d")})
cdi_df['End Date'] = pd.to_datetime(cdi_df['End Date'])
cdi_df.style.format({'End Date': lambda t: t.strftime("%Y-%m-%d")})

cdi_df = cdi_df[(cdi_df['Start Date'].isin(hac_df['Start Date'].unique())) | (cdi_df['End Date'].isin(hac_df['End Date'].unique()))]

###################  Merge CAUTI, CLABSI, MRSA, and CDI  ####################################

hai_df = cauti_df.merge(clabsi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(mrsa_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')
hai_df = hai_df.merge(cdi_df, on=['Facility ID', 'Start Date', 'End Date'], how='outer')


####################  Drop duplicate rows resulting from merger #############################

hai_df.drop_duplicates(inplace=True)


########################  Conversions to numeric  #############################################

features = ['CAUTI Urinary Catheter Days', 'CLABSI Device Days',
            'MRSA patient days', 'CDIFF patient days',
            'CLABSI Observed Cases', 'CLABSI Predicted Cases', 
            'MRSA Observed Cases', 'MRSA Predicted Cases', 
            'CDIFF Observed Cases', 'CDIFF Predicted Cases', 
            'CAUTI Observed Cases', 'CAUTI Predicted Cases', 
           ]

for f in features:
    hai_df[f] = hai_df[f].astype(str)
    hai_df[f] = hai_df[f].str.replace('*', '')
    hai_df[f] = hai_df[f].str.replace(' ', '')
    hai_df[f] = pd.to_numeric(hai_df[f], errors='coerce')
    
hai_df['Total device days'] = hai_df['CLABSI Device Days'] + hai_df['CAUTI Urinary Catheter Days']


########################  Reorder columns  #############################################

col_to_move = hai_df.pop('CAUTI Urinary Catheter Days')
hai_df.insert(hai_df.shape[1] - 2, 'CAUTI Urinary Catheter Days', col_to_move)

print(hai_df.shape)
hai_df.head()

(10023, 16)


,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days
0,030117,0.0,0.862,2019-01-01,2019-12-31,657.0,0.0,0.411,7022.0,0.0,0.153,6534.0,2.0,1658.0,1.713,2315.0
256,361311,0.0,0.021,2019-01-01,2019-12-31,6.0,0.0,0.002,1148.0,0.0,0.023,1148.0,1.0,31.0,0.254,37.0
512,520100,1.0,3.230,2019-01-01,2019-12-31,3241.0,0.0,2.219,17118.0,0.0,0.729,16219.0,2.0,4335.0,7.399,7576.0
768,520045,2.0,4.877,2019-01-01,2019-12-31,3849.0,2.0,3.451,34930.0,1.0,1.602,32193.0,9.0,4616.0,12.947,8465.0
1024,15004F,NaN,NaN,2019-01-01,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Filter HAI data on start dates and end dates that match those in the HAC file.

In [7]:
#########  Filter on start dates and end dates that match those in the HAC file  #######

hai_df = hai_df[(hai_df['Start Date'].isin(['2018-01-01', '2019-01-01'])) & (hai_df['End Date'].isin(['2018-12-31', '2019-12-31']))]

print(hai_df.shape)
print(hai_df['Start Date'].unique())
print(hai_df['End Date'].unique())

(10023, 16)
['2019-01-01T00:00:00.000000000' '2018-01-01T00:00:00.000000000']
['2019-12-31T00:00:00.000000000' '2018-12-31T00:00:00.000000000']


## Drop hospitals from HAI data that are not contained in HAC data.

In [8]:
# drop hospitals in HAI data that are not in HAC data
hai_df = hai_df[hai_df['Facility ID'].isin(hac_df['Facility ID'].unique())]

In [9]:
# hospitals in the hac data but not in the hai data ...
#tdf = hac_df.copy(deep=True) 
tdf = hac_df[~hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]
hac_df = hac_df[hac_df['Facility ID'].isin(hai_df['Facility ID'].unique())]

print(len(tdf['Facility ID'].unique()), 'hospitals in HAC dataset but not in HAI dataset\n')


tdf.head()

0 hospitals in HAC dataset but not in HAI dataset



,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Facility ID,Facility Name,Fiscal Year,End Date,Start Date,MRSA Footnote,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year


## Correct HAI file for non-duplicate rows having duplicate dates

**Problem:** Some rows for the same provider have duplicate measurement dates but different values for observed cases, predicted cases, etc. This results from each year having multiple (quarterly) files, the data within which can vary among files. Additionally, the most recent file for each year is not always the right file to use. 

**Need:** Since only one row can be used, we need to figure out which row should be used.

**Solution:** Select the row with the greatest totals for predicted cases for each HAI.

In [10]:
##############  Label rows that have duplicate dates (per provider) ####################
##############  For each provider with rows having duplicate dates,  ###################
###########  keep the last row (will have greatest number of total device days)  #######

hai_df['duplicated dates'] = hai_df.duplicated(subset=['Facility ID', 'Start Date', 'End Date'], keep=False)

hai_df.sort_values(by=['Facility ID', 'Start Date', 'End Date', 
                       'MRSA Predicted Cases', 'CAUTI Predicted Cases',  
                       'MRSA Observed Cases', 'CAUTI Observed Cases', 
                       'CDIFF Predicted Cases', 'CLABSI Predicted Cases',
                       'CDIFF Observed Cases', 'CLABSI Observed Cases',
                       ], inplace=True, ascending=False)

hai_df.drop_duplicates(subset=['Facility ID', 'Start Date', 'End Date'], inplace=True, keep='first')

print(hai_df.shape)
hai_df.head()

(6368, 17)


,Facility ID,CAUTI Observed Cases,CAUTI Predicted Cases,Start Date,End Date,CLABSI Device Days,CLABSI Observed Cases,CLABSI Predicted Cases,MRSA patient days,MRSA Observed Cases,MRSA Predicted Cases,CDIFF patient days,CDIFF Observed Cases,CAUTI Urinary Catheter Days,CDIFF Predicted Cases,Total device days,duplicated dates
721148,670143,NaN,NaN,2019-01-01,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
250166,670136,NaN,NaN,2019-01-01,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
300052,670135,NaN,NaN,2019-01-01,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
305776,670134,NaN,NaN,2019-01-01,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False
541077,670133,NaN,NaN,2019-01-01,2019-12-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,False


## Aggregate annual HAI data into biennial data

Purpose: Match the biennial measurement periods of HAC data


In [11]:
start_dates = hac_df['Start Date'].tolist()
end_dates = hac_df['End Date'].tolist()
prvdrs = hac_df['Facility ID'].tolist()

total_device_days = []

cauti_days = []
clabsi_days = []
mrsa_days = []
cdi_days = []

cauti_pred = []
clabsi_pred = []
mrsa_pred = []
cdi_pred = []

cauti_obs = []
clabsi_obs = []
mrsa_obs = []
cdi_obs = []

for i, start in enumerate(start_dates):
    end = end_dates[i]
    prvdr = prvdrs[i]
    
    tdf = hai_df[hai_df['Facility ID'] == prvdr]
    tdf = tdf[(tdf['Start Date'] == start) | (tdf['End Date'] == end)]
    
    if tdf.shape[0] == 1:
        pass
        #print('tdf.shape[0]:', 1)
        #print('hospital:', prvdr)
        #print(tdf['Start Date'].unique())
        #print(tdf['End Date'].unique())
        
    if tdf.shape[0] > 2:
        print('tdf.shape[0] = ', tdf.shape[0])
        for date_ in ['Start Date', 'End Date']:
            tdf['duplicated dates'] = tdf.duplicated(subset=[date_], keep=False)
            tdf.sort_values(by=[
                                'CAUTI Observed Cases', 'CLABSI Observed Cases',
                                'MRSA Observed Cases', 'CDIFF Observed Cases',
                                ], inplace=True, ascending=True)

            tdf.drop_duplicates(subset=[date_], inplace=True, keep='last')

        if tdf.shape[0] > 2:
            print('Error:')
            print("tdf.shape[0] > 2:", tdf.shape[0])
            print(start)
            print(end)
            print(tdf.head())
            break
        
    total_device_days.append(np.nansum(tdf['Total device days']))
    cauti_days.append(np.nansum(tdf['CAUTI Urinary Catheter Days']))
    clabsi_days.append(np.nansum(tdf['CLABSI Device Days']))
    mrsa_days.append(np.nansum(tdf['MRSA patient days']))
    cdi_days.append(np.nansum(tdf['CDIFF patient days']))
        
    cauti_pred.append(np.nansum(tdf['CAUTI Predicted Cases']))
    clabsi_pred.append(np.nansum(tdf['CLABSI Predicted Cases']))
    mrsa_pred.append(np.nansum(tdf['MRSA Predicted Cases']))
    cdi_pred.append(np.nansum(tdf['CDIFF Predicted Cases']))

    cauti_obs.append(np.nansum(tdf['CAUTI Observed Cases']))
    clabsi_obs.append(np.nansum(tdf['CLABSI Observed Cases']))
    mrsa_obs.append(np.nansum(tdf['MRSA Observed Cases']))
    cdi_obs.append(np.nansum(tdf['CDIFF Observed Cases']))
    

## Add HAI data to the HAC dataframe and save

In [12]:
hac_df['Total device days'] = total_device_days
hac_df['CAUTI Urinary Catheter Days'] = cauti_days
hac_df['CLABSI Device Days'] = clabsi_days
hac_df['MRSA patient days'] = mrsa_days
hac_df['CDI patient days'] = cdi_days

hac_df['CAUTI Observed Cases'] = cauti_obs
hac_df['CLABSI Observed Cases'] = clabsi_obs
hac_df['MRSA Observed Cases'] = mrsa_obs
hac_df['CDI Observed Cases'] = cdi_obs

hac_df['CAUTI Predicted Cases'] = cauti_pred
hac_df['CLABSI Predicted Cases'] = clabsi_pred
hac_df['MRSA Predicted Cases'] = mrsa_pred
hac_df['CDI Predicted Cases'] = cdi_pred

hac_df['CAUTI derived SIR'] = np.round(hac_df['CAUTI Observed Cases'] / hac_df['CAUTI Predicted Cases'],4)
hac_df['CLABSI derived SIR'] = np.round(hac_df['CLABSI Observed Cases'] / hac_df['CLABSI Predicted Cases'],4)
hac_df['MRSA derived SIR'] = np.round(hac_df['MRSA Observed Cases'] / hac_df['MRSA Predicted Cases'],4)
hac_df['CDI derived SIR'] = np.round(hac_df['CDI Observed Cases'] / hac_df['CDI Predicted Cases'],4)

print('hac_df.shape:', hac_df.shape)
print(len(hac_df['Facility ID'].unique()), 'hospitals in 2021 HACRP')
#hac_df.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/merged_HAI_HAC_2021.pkl', protocol=5)
hac_df.head()



hac_df.shape: (3204, 43)
3204 hospitals in 2021 HACRP


,CAUTI Footnote,CAUTI W Z Score,CDI Footnote,CDI W Z Score,CLABSI Footnote,CLABSI W Z Score,Facility ID,Facility Name,Fiscal Year,End Date,Start Date,MRSA Footnote,MRSA W Z Score,PSI-90 End Date,PSI-90 Footnote,PSI-90 Start Date,PSI-90 W Z Score,Payment Reduction,Payment Reduction Footnote,SSI Footnote,SSI W Z Score,State,Total HAC Footnote,Total HAC Score,file_month,file_year,Total device days,CAUTI Urinary Catheter Days,CLABSI Device Days,MRSA patient days,CDI patient days,CAUTI Observed Cases,CLABSI Observed Cases,MRSA Observed Cases,CDI Observed Cases,CAUTI Predicted Cases,CLABSI Predicted Cases,MRSA Predicted Cases,CDI Predicted Cases,CAUTI derived SIR,CLABSI derived SIR,MRSA derived SIR,CDI derived SIR
0,NaN,-0.4612,NaN,-0.3173,NaN,0.6029,010001,SOUTHEAST ALABAMA MEDICAL CENTER,2021,2019-12-31,2018-01-01,NaN,-0.7886,06/30/2019,NaN,07/01/2017,-0.3375,No,NaN,NaN,-1.0731,AL,NaN,-0.3958,10,2021,41201.0,26193.0,15008.0,198371.0,198371.0,18.0,14.0,5.0,74.0,34.568,14.573,13.323,146.527,0.5207,0.9607,0.3753,0.5050
2130,NaN,-0.7165,NaN,0.0407,NaN,-0.6679,360051,MIAMI VALLEY HOSPITAL,2021,2019-12-31,2018-01-01,NaN,0.5558,06/30/2019,NaN,07/01/2017,0.3532,No,NaN,NaN,-0.1671,OH,NaN,-0.1003,10,2021,104878.0,46558.0,58320.0,461679.0,419334.0,29.0,22.0,42.0,198.0,72.637,62.941,38.299,321.602,0.3992,0.3495,1.0966,0.6157
2131,NaN,-1.1247,NaN,0.9986,NaN,0.6632,360054,HOLZER MEDICAL CENTER,2021,2019-12-31,2018-01-01,NaN,0.8686,06/30/2019,NaN,07/01/2017,1.1350,Yes,NaN,NaN,0.4893,OH,NaN,0.5050,10,2021,7080.0,4879.0,2201.0,38938.0,36421.0,1.0,2.0,3.0,22.0,4.905,2.021,2.372,24.108,0.2039,0.9896,1.2648,0.9126
2132,NaN,0.5122,NaN,0.0729,NaN,0.7360,360055,TRUMBULL REGIONAL MEDICAL CENTER,2021,2019-12-31,2018-01-01,NaN,-0.2616,06/30/2019,NaN,07/01/2017,0.9731,Yes,NaN,NaN,2.0795,OH,NaN,0.6853,10,2021,22302.0,13043.0,9259.0,79583.0,78963.0,17.0,10.0,3.0,33.0,17.236,9.754,4.560,52.728,0.9863,1.0252,0.6579,0.6259
2133,NaN,-0.9070,NaN,0.0407,NaN,-0.9112,360056,MERCY HEALTH - FAIRFIELD HOSPITAL,2021,2019-12-31,2018-01-01,NaN,-0.6657,06/30/2019,NaN,07/01/2017,1.0710,No,NaN,NaN,-1.1064,OH,NaN,-0.4131,10,2021,29154.0,17256.0,11898.0,112488.0,105893.0,4.0,2.0,2.0,46.0,12.986,8.594,4.535,74.710,0.3080,0.2327,0.4410,0.6157


## Generate Winsorized z-scores

In [13]:
hais = ['CAUTI', 'CLABSI', 'MRSA', 'CDI']

df_yr = hac_df.copy(deep=True)
    
for i, hai in enumerate(hais):
    tdf2 = df_yr[~df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18', 
                                              5, '5', ' 5', '5 ',
                                              4, '4', ' 4', '4 ',
                                              ])]
      
    reported_winZ = tdf2[hai + ' W Z Score'].tolist()
    sirs = tdf2[hai + ' derived SIR'].tolist()
    tdf2[hai + ' derived Winsorized SIR'] = Winsorize_it(sirs, reported_winZ)
    tdf2[hai + ' derived W Z Score'] = ZScore_it(tdf2[hai + ' derived Winsorized SIR'], reported_winZ)
    
    # Assign maximum WinZ scores to hospitals with HAI footnote 18 
    maxWinZ = np.nanmax(tdf2[hai + ' derived W Z Score'])
    tdf3 = df_yr[df_yr[hai + ' Footnote'].isin([18, '18', '18 ', ' 18'])]
    tdf3[hai + ' derived Winsorized SIR'] = [np.nan]*tdf3.shape[0]
    tdf3[hai + ' derived W Z Score'] = [maxWinZ]*tdf3.shape[0]
    
    tdf4 = df_yr[df_yr[hai + ' Footnote'].isin([5, '5', ' 5', '5 ', 4, '4', ' 4', '4 '])]
    tdf4[hai + ' derived Winsorized SIR'] = [np.nan]*tdf4.shape[0]
    tdf4[hai + ' derived W Z Score'] = [np.nan]*tdf4.shape[0]
    
    df_yr = pd.concat([tdf2, tdf3, tdf4], axis=0)

del tdf2
del tdf3

display_df = df_yr.copy(deep=True)
items = ['file_year', 'HAI Measures End Date', 'HAI Measures Start Date',
         'CAUTI Footnote', 'CAUTI W Z Score', 'CAUTI derived W Z Score', 'CAUTI derived Winsorized SIR',
         'CDI Footnote', 'CDI W Z Score', 'CDI derived W Z Score',
         'CLABSI Footnote', 'CLABSI W Z Score', 'CLABSI derived W Z Score',
         'MRSA Footnote', 'MRSA W Z Score', 'MRSA derived W Z Score',
         #'Total HAC Footnote', 'Total HAC Score',
         ]


display_df = display_df.filter(items=items)
#display_df = display_df[display_df['CAUTI W Z Score'].isin([np.nan, float("NaN")])]
#display_df.sort_values(by='CAUTI derived W Z Score', inplace=True, ascending=False)
display_df.head(20)

,file_year,CAUTI Footnote,CAUTI W Z Score,CAUTI derived W Z Score,CAUTI derived Winsorized SIR,CDI Footnote,CDI W Z Score,CDI derived W Z Score,CLABSI Footnote,CLABSI W Z Score,CLABSI derived W Z Score,MRSA Footnote,MRSA W Z Score,MRSA derived W Z Score
0,2021,NaN,-0.4612,-0.459863,0.520700,NaN,-0.3173,-0.318733,NaN,0.6029,0.603751,NaN,-0.7886,-0.787765
2130,2021,NaN,-0.7165,-0.713737,0.399200,NaN,0.0407,0.038663,NaN,-0.6679,-0.668718,NaN,0.5558,0.555731
2131,2021,NaN,-1.1247,-1.121816,0.203900,NaN,0.9986,0.997207,NaN,0.6632,0.663918,NaN,0.8686,0.869021
2132,2021,NaN,0.5122,0.513008,0.986300,NaN,0.0729,0.071593,NaN,0.7360,0.738034,NaN,-0.2616,-0.261393
2133,2021,NaN,-0.9070,-0.904299,0.308000,NaN,0.0407,0.038663,NaN,-0.9112,-0.911886,NaN,-0.6657,-0.665391
2135,2021,NaN,-1.2921,-1.289393,0.123700,NaN,0.0762,0.076113,NaN,-0.3122,-0.312294,NaN,1.3341,1.335231
2136,2021,NaN,-0.2728,-0.271181,0.611000,NaN,-0.3012,-0.302914,NaN,-0.6180,-0.618752,NaN,-0.7569,-0.757404
2137,2021,NaN,-0.3460,-0.344313,0.576000,NaN,-0.0399,-0.042373,NaN,2.2542,2.248415,NaN,-1.4868,-1.486800
2138,2021,NaN,-0.8902,-0.886747,0.316400,NaN,-0.9462,-0.948615,NaN,-1.1587,-1.159427,NaN,0.0028,0.002911
2139,2021,NaN,-0.3377,-0.118647,0.684000,NaN,0.6696,0.669836,NaN,-0.3351,-0.668510,NaN,-0.5688,-0.567791


In [14]:

features = ['CAUTI derived SIR', 'CAUTI W Z Score', 'CAUTI derived W Z Score',
            'CDI derived SIR', 'CDI W Z Score', 'CDI derived W Z Score',
            'CLABSI derived SIR', 'CLABSI W Z Score', 'CLABSI derived W Z Score',
            'MRSA derived SIR', 'MRSA W Z Score', 'MRSA derived W Z Score', 
            'SSI W Z Score', 'PSI-90 W Z Score', 'Total HAC Score']

for f in features:
    df_yr[f] = df_yr[f].astype(str)
    df_yr[f] = pd.to_numeric(df_yr[f], errors='coerce')
    

for hai in hais:
    print(hai)
    
    ls1 = df_yr[hai + ' W Z Score'].tolist()
    ls2 = df_yr[hai + ' derived W Z Score'].tolist()
    final_ls = []
    
    for i, val in enumerate(ls1):
        if np.isnan(val) == False and np.isnan(ls2[i]) == True:
            final_ls.append(ls2[i])
        elif np.isnan(val) == False and np.isnan(ls2[i]) == False:
            final_ls.append(ls2[i])
        elif np.isnan(val) == True and np.isnan(ls2[i]) == False:
            final_ls.append(ls2[i])
        elif np.isnan(val) == True and np.isnan(ls2[i]) == True:
            final_ls.append(ls2[i])
        else:
            final_ls.append(ls2[i])
            
    df_yr[hai + ' derived W Z Score'] = final_ls


CAUTI
CLABSI
MRSA
CDI


## Attempt to reproduce HAC scores for 2021

In [15]:
print('Results from attempting to reproduce Yes/No penalty assignments:\n')
print('Excluded from results below:')
print('1. MD hospitals')
print('2. Hospitals with payment reduction values other than Yes or No\n')

df_yr.dropna(how='all', axis=1, inplace=True)

holdout_df = df_yr[(df_yr['State'] == 'MD') | ~df_yr['Payment Reduction'].isin(['Yes', 'No']) | (df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]
df_yr = df_yr[(df_yr['State'] != 'MD') & (df_yr['Payment Reduction'].isin(['Yes', 'No'])) & (~df_yr['Total HAC Score'].isin([float("NaN"), np.nan]))]

hac_scores = []
ct1 = 0
ct2 = 0


for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    d = 0
        
    w_ls = []
    sum_ls = []

    # Use original scores for all HAIs to test whether can penalties be reproduced when 
    # using data from the HACRP files
    #m_ls = ['CDI W Z Score', 'CAUTI W Z Score', 'CLABSI W Z Score', 'MRSA W Z Score', 'SSI W Z Score']

    # Use original SSI scores but derived scores for CDI, CAUTI, CLABSI, and MRSA for actual results
    m_ls = ['PSI-90 W Z Score', 'CDI derived W Z Score', 'CAUTI derived W Z Score', 'CLABSI derived W Z Score', 'MRSA derived W Z Score', 'SSI W Z Score']
                
    s = 0
    w = 0
    for m in m_ls:
        v = tdf[m].tolist()
        if len(list(set(v))) > 1:
            print('len(list(set(v))) > 1')
            sys.exit()

        v = tdf[m].iloc[0]

        if np.isnan(v) == False: 
            s += v
            w += 1

    if w == 0:
        d = np.nan
    else:
        d = s/w
        
    hac_scores.append(d)
    state = tdf['State'].iloc[0]
    
        
df_yr['Total HAC Score (derived)'] = hac_scores
print(df_yr.shape[0], 'hospitals in hac_df')


Results from attempting to reproduce Yes/No penalty assignments:

Excluded from results below:
1. MD hospitals
2. Hospitals with payment reduction values other than Yes or No

3105 hospitals in hac_df


In [16]:
p75 = np.nanpercentile(df_yr['Total HAC Score'], 75)
p75 = 0.3383
print('p75:', p75)

pr = []
for hosp in df_yr['Facility ID'].tolist():
    tdf = df_yr[df_yr['Facility ID'] == hosp]

    score = tdf['Total HAC Score'].iloc[0]

    if np.isnan(score) == True:
        pr.append('No')
    elif score <= p75:
        pr.append('No')
    elif score > p75:
        pr.append('Yes')
    else:
        print('This score is an error:', score)
        sys.exit()

df_yr['Payment Reduction (derived)'] = pr
    
o_list = df_yr['Payment Reduction'].tolist()
d_list = df_yr['Payment Reduction (derived)'].tolist()

same = 0
diff = 0
res_ls = []
for i, o in enumerate(o_list):
    if o == d_list[i]:
        same += 1
        res_ls.append(1)
    else:
        diff += 1
        res_ls.append(0)
            
df_yr['Payment Reduction Reproduced?'] = res_ls
    
print(same, "Penalty assignments were reproduced")
print(diff, "Penalty assignments were not reproduced")
print(str(np.round(100 * same/(same+diff), 2)) + '% penalty assignments were reproduced\n')
    
df_yr['HAC delta'] = df_yr['Total HAC Score'] - df_yr['Total HAC Score (derived)']
df_yr['CDI delta'] = df_yr['CDI W Z Score'] - df_yr['CDI derived W Z Score']
df_yr['MRSA delta'] = df_yr['MRSA W Z Score'] - df_yr['MRSA derived W Z Score']
df_yr['CAUTI delta'] = df_yr['CAUTI W Z Score'] - df_yr['CAUTI derived W Z Score']
df_yr['CLABSI delta'] = df_yr['CLABSI W Z Score'] - df_yr['CLABSI derived W Z Score']


p75: 0.3383
3105 Penalty assignments were reproduced
0 Penalty assignments were not reproduced
100.0% penalty assignments were reproduced



In [17]:
ls1 = list(df_yr)
ls2 = list(holdout_df)
ls = list(filter(lambda x:x in ls1, ls2))
print(df_yr.shape)
print(holdout_df.shape)
df_yr = df_yr.merge(holdout_df, how='outer', on=ls)
print(df_yr.shape)


(3105, 59)
(99, 51)
(3204, 59)


In [18]:
tdf1 = df_yr[df_yr['Payment Reduction'] == 'Yes']
tdf2 = df_yr[df_yr['Payment Reduction'].isin(['Yes', 'No'])]
print(tdf1.shape[0]/tdf2.shape[0])

tdf1 = df_yr[df_yr['Payment Reduction'] == 'No']
print(tdf1.shape[0]/tdf2.shape[0])

print(df_yr.shape[0])
print(len(df_yr['Facility ID'].unique()))

df_yr.to_pickle('~/GitHub/HACRP-HAIs/data/merged_HAC_HAI/HAI_HAC_2021.pkl', protocol=5)


0.24509183027232426
0.7549081697276757
3204
3204
